In [9]:
from neptune import Project
import keys
import pandas as pd
import zentables as zen

project = Project(project="VRNET/MetaINR", api_token=keys.NEPTUNE_API_TOKEN, mode="read-only")
exps = project.fetch_runs_table(columns=["sys/creation_time", "sys/running_time", "sys/description", "average_PSNR", "dataset", "target"]).to_pandas().sort_values(by="sys/creation_time", ascending=False)

notes = [
    "MetaINR 5e-5, 1e-4 pretrian 8 step 1e-5 adapt",
    "VP Baseline 5e-5, 1e-4 pretrian 8 step 1e-5 adapt",
    "INR50 1e-5 lr"
]

datasets = [
    "vorts",
    "half-cylinder",
    "earthquake",
    "combustion",
    "fivejet",
    "ionization",
    "supernova",
    "tangaroa",
]

# create a table of dataset on columns and target on rows
table = {}
activated_context = None
active_length = 0
for index, row in exps.iterrows():
    note = row["sys/description"]
    dataset = row["dataset"]
    if note in notes:
        if note != activated_context:
            print(note)
            activated_context = note
            active_length = 0
        if activated_context.split(" ")[0] not in table:
            table[activated_context.split(" ")[0]] = {}
    if active_length < 8:
        active_length += 1
    else:
        break
    runtime_s = row["sys/running_time"]
    runtime_h = int(runtime_s // 3600)
    runtime_m = int((runtime_s % 3600) // 60)
    runtime_s = int(runtime_s % 60)
    # if no hour, display min and sec, otherwise display hour and min
    time_display = f"{runtime_h}h {runtime_m}m" if runtime_h > 0 else f"{runtime_m}m {runtime_s}s"
    table[activated_context.split(" ")[0]][dataset] = f"{row['average_PSNR']:.2f} ({time_display})"

# export to xlsx
df = pd.DataFrame(table)
df.to_excel("table.xlsx")

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/VRNET/MetaINR/


Fetching table...: 0 [00:00, ?/s]

MetaINR 5e-5, 1e-4 pretrian 8 step 1e-5 adapt
VP Baseline 5e-5, 1e-4 pretrian 8 step 1e-5 adapt
INR50 1e-5 lr
